In [2]:
import numpy as np
import numba as nb
import Ivvsc
import importlib
import time

In [11]:
filename = "Run35.npz"

In [12]:
data = np.load(filename, allow_pickle=True)
settings = data['settings'].item()
raw = data['raw'].item()
prob = data['prob'].item()

In [13]:
N = settings['N'] + 1
e_max = settings['eps_max'] * settings['T']
B = 2

p = np.linspace(0, e_max, N)
delta_p = p[1] - p[0]

In [14]:
def calc_c(data, dN, p, e_max, B, dp, output_filename = "data", verbose=True):
    C_full = np.zeros_like(data)
    
    N_total = len(data)
    
    N_steps = N_total // dN
    
    time_array = np.zeros(N_steps)
    
    for i in range(N_steps):
        beg = time.time()
        for j in range(dN):
            index = i * dN + j
            if verbose:
                print("Time step {} / {}".format(index+1, N_total))
            C_full[index, :, :] = Ivvsc.C_Ivvsc(data[index,:,:], p, e_max, B, dp)
        np.savez(output_filename + "-" + str(i), C = C_full[:(i+1)*dN,:,:], p = p)
        print("File {}-{}.npz number {} of {}.".format(output_filename, i, i+1, N_steps))
        time_array[i] = (time.time() - beg) 
        time_string = "{:.1f}\% done. total time: {:.2f} hrs ({:.1f} min for last file)".format(100*(index+1)/N_total, np.sum(time_array)/60/60, time_array[i]/60)
        print(time_string)
        
    if index != len(data) - 1:
        i = N_steps
        for k in range(index+1, len(data)):
            C_full[k, :, :] = Ivvsc.C_Ivvsc(data[k,:,:], p, e_max, B, dp)
            
        
    np.savez(output_filename, C = C_full, p = p)
            
    return

In [15]:
def calc_c_fix_end(data, C_short, p, e_max, B, dp, output_filename = "data"):
    C_full = np.zeros_like(data)
    
    if len(data) == len(C_short):
        print("Already done.")
        return
    
    C_full[:len(C_short), :, :] = C_short[:,:,:]
    for k in range(len(C_short), len(data)):
        C_full[k, :, :] = Ivvsc.C_Ivvsc(data[k,:,:], p, e_max, B, dp)
        
    np.savez(output_filename, C = f_full, p = p)
    
    return

In [10]:
%%time
calc_c(raw['nu3D'], 75, p, e_max, B, delta_p, output_filename="test")

Time step 1 / 759
Time step 2 / 759
Time step 3 / 759
Time step 4 / 759
Time step 5 / 759
Time step 6 / 759
Time step 7 / 759
Time step 8 / 759
Time step 9 / 759
Time step 10 / 759
Time step 11 / 759
Time step 12 / 759
Time step 13 / 759
Time step 14 / 759
Time step 15 / 759
Time step 16 / 759
Time step 17 / 759
Time step 18 / 759
Time step 19 / 759
Time step 20 / 759
Time step 21 / 759
Time step 22 / 759
Time step 23 / 759
Time step 24 / 759
Time step 25 / 759
Time step 26 / 759
Time step 27 / 759
Time step 28 / 759
Time step 29 / 759
Time step 30 / 759
Time step 31 / 759
Time step 32 / 759
Time step 33 / 759
Time step 34 / 759
Time step 35 / 759
Time step 36 / 759
Time step 37 / 759
Time step 38 / 759
Time step 39 / 759
Time step 40 / 759
Time step 41 / 759
Time step 42 / 759
Time step 43 / 759
Time step 44 / 759
Time step 45 / 759
Time step 46 / 759
Time step 47 / 759
Time step 48 / 759
Time step 49 / 759
Time step 50 / 759
Time step 51 / 759
Time step 52 / 759
Time step 53 / 759
Ti

In [16]:
c_mat = np.load("Run35Cbar-7.npz", allow_pickle=True)
c_s = c_mat['C']

In [17]:
calc_c_fix_end(raw['nu3D'], c_s, p, e_max, B, delta_p, output_filename="Run35Cbar")

600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
